In [1]:
import cuvis_ai
import cuvis
import yaml
from EfficientAD_torch_model import EfficientAdModel
import torch
import numpy as np
from torchvision.transforms import v2
import cv2 as cv
from pathlib import Path

In [2]:
def remove_prefix(s, prefix):
    if s.startswith(prefix):
        return s[len(prefix):]
    return s

In [3]:
CONF ='example_train_config.yaml'
WEIGHTS = '../data/checkpoints/Testrun/Testrun-epoch=00-val_im/AU-ROC=0.76.ckpt'
MESU = '../data/test_data/test_cube.cu3s'

## Load Model

First we create a cuvis.ai Graph using some the config parameters from our training.

In [4]:
with open(CONF, 'r') as f:
    config = yaml.safe_load(f)

graph = cuvis_ai.pipeline.Graph('TestAD')

modelNode = cuvis_ai.node.wrap.make_node(EfficientAdModel)(
    criterion=torch.nn.NLLLoss, teacher_out_channels=384, model_size='medium', in_channels=6)

## Create Pipeline

Using some transformation nodes, a pipeline for data pre- and postprocessing is created

In [5]:
CenterCrop = cuvis_ai.node.wrap.make_node(v2.CenterCrop)

Resize = cuvis_ai.node.wrap.make_node(v2.Resize)

Pad = cuvis_ai.node.wrap.make_node(v2.Pad)

graph = graph >> CenterCrop((2100, 4600)) >> Resize(
    (1050, 2300)) >> modelNode >> Pad(padding=75)

## Load checkpoint

We can easily load the model we previously trained using `train.py` into our modelNode

In [6]:
checkpoint = torch.load(WEIGHTS, weights_only=True)

#TODO: check if i need to remove prefixes in this case
state_dict = {remove_prefix(
    k, 'model.'): v for k, v in checkpoint['state_dict'].items()}

modelNode.model.load_state_dict(state_dict)
modelNode.initialized = True

## Save cuvis.ai graph

When saving the graph to disk, the whole pipeline and model code will be saved as well.

In [7]:
graph.save_to_file('effADTuesdayDataset')

Cant find class 'ValueError'
Cant find class 'super'
Cant find class 'self'
Cant find class 'super'
Cant find class 'self'
Cant find class 'super'
Cant find class 'self'
Cant find class 'x'
Cant find class 'any'
Cant find class 'super'
Cant find class 'self'
Cant find class 'batch'
Cant find class 'super'
Cant find class 'self'
Cant find class 'x'
Cant find class 'value'
Cant find class 'p_dic'
Cant find class 'super'
Cant find class 'self'
Cant find class 'x'
Project saved to effADTuesdayDataset


## Load cuvis.ai graph from file

We can now load the zip file again into any project without having to worry about the model class being available there.

In [8]:
loaded = cuvis_ai.pipeline.Graph.load_from_file(
    'effADTuesdayDataset.zip')

Re-initializing module because the following parameters were re-set: module__in_channels, module__model_size, module__teacher_out_channels.
Re-initializing criterion.
Re-initializing optimizer.


## Infer one cube

Using the forward function of our Graph we can now easily infer any cubert SessionFile that has the same dimensions as the ones we trained our model with. 

In [11]:
num_samples = 1
input_mesu = cuvis.SessionFile(str(MESU)).get_measurement(0)
proc = cuvis.ProcessingContext(input_mesu)
proc.apply(input_mesu)
input_data = input_mesu.data["cube"].array
input_data = input_data[:, :, [2, 1, 0, 5, 4, 3]] # convert BGR to RGB
#expected = cv.imread(EXPECTED_PRED)

normalized_data = np.expand_dims(input_data, 0)
normalized_data = normalized_data.astype(np.float32)
output = loaded.forward(normalized_data)

--- 54.659531593322754 seconds ---


## Display the result


In [12]:
cv.imshow("output", output[0])
cv.imshow("input", cv.normalize(input_data[:,:,:3], None, 0,255,cv.NORM_MINMAX,0))

#cv.imshow("expected", expected)

cv.waitKey(-1)
cv.destroyAllWindows()